# ABIO558 Assignment Check-In

### Project Goals:
1. Write markdown text describing the question and outlining your plan for analysis
2. Insert a snippet of code to load data
3. print() or visualization a command to view the data

## Goal 1: Outline your plan for analysis

In [ ]:
# Question: How does EspM regulate whiB6 expression in the recipient M. smegmatis strain MKD8?
# Data we have: RNA-seq data of the WT and espM deletion (KO) strains of MKD8
# Project Goal: To identify genes that are expressed differently when EspM is missing - want to make tables of significant changes in gene expression
# --> also want to look at the effect of espM deletion on the expression of other genes in the genome

# Plan for Analysis (in summary): Analyze MKD8 WT vs espM KO RNASeq data to determine gene expression differences
# --> conditions are WT and deletion of the espM gene. Looking for significant changes
# in expression of genes in the whiB6 regulon (and/or anywhere else in the genome)

# Outline:

# Pull out raw count data of each gene for each of the 4 samples

# Normalize raw count data and calculate p-values

# Bonferroni/Multiple testing correction
# Note: this gives corrected p-values to account for false significance

# Find ratio of KO/WT

# Make a log2 expression of the ratio

# logfold2 and then transform that into log2

# generate Volcano and MA plots for visualization


## Goal 2: Make Code to Load Data

### Step 1: Pull out the correct data columns

1) Import libraries
2) Find file path - check that it's correct
3) Load data with pandas
4) Pull out raw count columns for each of the 4 samples (note: rename columns - Biorep 1 = OLD, Biorep 2 = NEW)

In [2]:
# 1. Import Libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import scanpy
import anndata
import seaborn as sns
import skimage
from skimage.io import imread
from scipy import stats

In [3]:
# 2. Find file path

data_directory = 'C:/Users/denis/Downloads/'
filepath_mkd8 = os.path.join(data_directory, 'MKD8_KO_Rockhopper_data.csv')
os.path.exists(filepath_mkd8)

True

### Goal 3: Visualize a command

In [32]:
# 3. Load data with pandas

reads_df = pd.read_csv(filepath_mkd8, index_col=0)
reads_df.head()

,Translation Start,Translation Stop,Transcription Stop,Strand,Name,Synonym,Product,Raw Counts MKD8-oldmycospike,Normalized Counts MKD8-oldmycospike,RPKM MKD8-oldmycospike,...,pValue MKD8-oldmycospike vs KOespM-OLDmycospike,qValue MKD8-oldmycospike vs KOespM-OLDmycospike,pValue MKD8-oldmycospike vs KOespM-NEWmycospike,qValue MKD8-oldmycospike vs KOespM-NEWmycospike,pValue MKD8-NEWmycospike vs KOespM-OLDmycospike,qValue MKD8-NEWmycospike vs KOespM-OLDmycospike,pValue MKD8-NEWmycospike vs KOespM-NEWmycospike,qValue MKD8-NEWmycospike vs KOespM-NEWmycospike,pValue KOespM-OLDmycospike vs KOespM-NEWmycospike,qValue KOespM-OLDmycospike vs KOespM-NEWmycospike
Transcription Start,,,,,,,,,,,,,,,,,,,,,
NaN,1.0,1515.0,NaN,+,dnaA,MSMEI_0001,chromosomal replication initiator protein dnaA,4093.0,466488.0,232.0,...,0.047732,1.0,0.232398,1.0,0.048318,1.0,0.069294,1.0,7.290000e-21,4.220000e-19
NaN,1454.0,2071.0,2071.0,+,-,MSMEI_0002,hypothetical protein,1969.0,224415.0,274.0,...,0.658421,1.0,0.715949,1.0,0.665426,1.0,0.722310,1.0,1.092048e-02,5.016868e-02
2072.0,2109.0,3302.0,3302.0,+,dnaN,MSMEI_0003,DNA polymerase III subunit beta,13295.0,1515174.0,959.0,...,0.169175,1.0,0.213948,1.0,0.879587,1.0,0.287741,1.0,4.100000e-13,2.510000e-12
3303.0,3331.0,4224.0,4224.0,+,-,MSMEI_0004,6-phosphogluconate dehydrogenase,8056.0,918148.0,776.0,...,0.959884,1.0,0.349188,1.0,0.585358,1.0,0.751938,1.0,1.660000e-11,9.370000e-11
4225.0,4234.0,5388.0,NaN,+,recF,MSMEI_0005,DNA replication and repair protein RecF,3001.0,342014.0,223.0,...,0.428367,1.0,0.319915,1.0,0.677851,1.0,0.699315,1.0,2.010000e-17,2.650000e-16


In [34]:
# 4. extract the raw count columns

reads_df_2 = pd.read_csv(filepath_mkd8, usecols=['Raw Counts MKD8-oldmycospike', 'Raw Counts MKD8-NEWmycospike', 'Raw Counts KOespM-OLDmycospike', 'Raw Counts KOespM-NEWmycospike', 'Synonym'])
print(reads_df_2)

# do df.set index to name of synonym gene names

            Synonym  Raw Counts MKD8-oldmycospike  \
0        MSMEI_0001                        4093.0   
1        MSMEI_0002                        1969.0   
2        MSMEI_0003                       13295.0   
3        MSMEI_0004                        8056.0   
4        MSMEI_0005                        3001.0   
...             ...                           ...   
6886  predicted RNA                          75.0   
6887     MSMEI_6749                        4085.0   
6888     MSMEI_6750                        9695.0   
6889     MSMEI_6751                        7317.0   
6890     MSMEI_6752                        2700.0   

      Raw Counts MKD8-NEWmycospike  Raw Counts KOespM-OLDmycospike  \
0                           3700.0                          1442.0   
1                           1744.0                           749.0   
2                           9364.0                          4701.0   
3                           5592.0                          4026.0   
4            

In [35]:
# Note: You can transpose (use ".T") to do scanpy functions - this inverts the rows and columns

In [ ]:
# useful page for later in project: https://igb.mit.edu/mini-courses/python/data-processing-with-python/seaborn/visualizing-rnaseq-data